In [4]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [9]:
#确定数据
"""
假设有500条数据，20个特征，标签分为3类。
神经网络：3层
  第一层：13个神经元，激活函数relu
  第二层：8个神经元，激活函数sigmoid
  第三层为输出层

"""
torch.manual_seed(420)
X = torch.rand((500,20),dtype=torch.float32)
y = torch.randint(low=0,high=3,size=(500,1),dtype=torch.float32)

In [10]:
# torch.nn -> nn.Module需要的所有层，nn.functional需要的所有函数

In [26]:
# class 子类名(要继承父类的名字)
class Model(nn.Module):
    def __init__(self,in_features=10,out_features=2):
        """
        in_features：表示输入神经网络的特征数（输入层上的神经元数目）
        out_features：表示神经网络的输出数目（输出层上的神经元数目）
        """
        # 默认继承父类除了__init__之外的属性，如果要继承__init__内的属性则要使用 super(x，self).替换对象y   找到子类x的父类，替换 self也就是子类x 的对象y
        super(Model,self).__init__()
        # nn.Linear(上一层的神经元个数，本层的神经元个数，是否有偏差)
        self.linear1 = nn.Linear(in_features,13,bias=True) 
        self.linear2 = nn.Linear(13,8,bias=True)
        self.output = nn.Linear(8,out_features,bias=True)
        
    def forward(self,x): # 神经网络前向传播
        z1 = self.linear1(x)
        sigma1 = torch.relu(z1)
        z2 = self.linear2(sigma1)
        sigma2 = torch.sigmoid(z2)
        z3 = self.output(sigma2)
        sigma3 = F.softmax(z3,dim=1)
        
        return sigma3

In [27]:
X.shape

torch.Size([500, 20])

In [28]:
# y的类别
y.unique()

tensor([0., 1., 2.])

In [29]:
input_ = X.shape[1]
output_ = len(y.unique())

In [30]:
# 实例化神经网络
torch.random.manual_seed(420) # 在调用nn.Module时会随机生成特征权重W和b，因此提前随机生成
net = Model(in_features=input_, out_features=output_)

In [31]:
net.forward(X)

tensor([[0.4140, 0.3496, 0.2365],
        [0.4210, 0.3454, 0.2336],
        [0.4011, 0.3635, 0.2355],
        ...,
        [0.4196, 0.3452, 0.2352],
        [0.4153, 0.3455, 0.2392],
        [0.4153, 0.3442, 0.2405]], grad_fn=<SoftmaxBackward>)

In [32]:
net.forward(X).shape # 500个样本，3个标签的概率

torch.Size([500, 3])

In [33]:
# 执行Model里除了__init__外所有的函数
net(X)

tensor([[0.4140, 0.3496, 0.2365],
        [0.4210, 0.3454, 0.2336],
        [0.4011, 0.3635, 0.2355],
        ...,
        [0.4196, 0.3452, 0.2352],
        [0.4153, 0.3455, 0.2392],
        [0.4153, 0.3442, 0.2405]], grad_fn=<SoftmaxBackward>)

In [43]:
# w * X        特征X(500,20) ->行列转为 (20,500)

In [36]:
# 第一层的权重
# net.linear1.weight 
net.linear1.weight.shape

torch.Size([13, 20])

In [40]:
# w (13,20) * (20,500) -> (13,500)

In [37]:
# 第二层的权重
# net.linear2.weight 
net.linear2.weight.shape

torch.Size([8, 13])

In [41]:
# w (8,13) * (13,500) -> (8,500)

In [38]:
# 输出层的权重
# net.output.weight 
net.output.weight.shape

torch.Size([3, 8])

In [42]:
# w (3,8) * (8,500) -> (3,500)     输出(3,500) -> 行列转为 （500，3）

In [46]:
net.linear1.bias

Parameter containing:
tensor([ 1.3508e-01,  1.5439e-01, -1.9350e-01, -6.8777e-02,  1.3787e-01,
        -1.8474e-01,  1.2763e-01,  1.8031e-01,  9.5152e-02, -1.2660e-01,
         1.4317e-01, -1.4945e-01,  3.4258e-05], requires_grad=True)

In [47]:
net.linear2.bias

Parameter containing:
tensor([ 0.0900, -0.0597,  0.0268, -0.0173,  0.0065,  0.0228, -0.1408,  0.1188],
       requires_grad=True)

In [48]:
net.output.bias

Parameter containing:
tensor([-0.0156, -0.0730, -0.2637], requires_grad=True)

In [49]:
# 属性的继承

In [50]:
net.training #是否要用于训练

True

In [51]:
# 方法的继承

In [52]:
net.cuda() #将整个网络转移到GPU上运行

AssertionError: Torch not compiled with CUDA enabled

In [53]:
net.cpu() #将整个网络转移到CPU上运行

Model(
  (linear1): Linear(in_features=20, out_features=13, bias=True)
  (linear2): Linear(in_features=13, out_features=8, bias=True)
  (output): Linear(in_features=8, out_features=3, bias=True)
)

In [54]:
net.apply() #对神经网络所有的层，init函数中所有的对象都执行同样的操作

TypeError: apply() missing 1 required positional argument: 'fn'

In [55]:
def initial_0(m):
    print(m)
    if type(m) == nn.Linear:
        m.weight.data.fill_(0)
        print(m.weight)
        
net.apply(initial_0)

Linear(in_features=20, out_features=13, bias=True)
Parameter containing:
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.

Model(
  (linear1): Linear(in_features=20, out_features=13, bias=True)
  (linear2): Linear(in_features=13, out_features=8, bias=True)
  (output): Linear(in_features=8, out_features=3, bias=True)
)

In [56]:
net.parameters() # 求得的所有w和b

<generator object Module.parameters at 0x7f9d834f7890>

In [57]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0